In [ ]:
import numpy as np # linear algebra
import pandas as pd
import sqlite3

df_olist_customers = pd.read_csv('/content/olist_customers_dataset.csv')
df_olist_sellers = pd.read_csv('/content/olist_sellers_dataset.csv')
df_olist_order_reviews= pd.read_csv('/content/olist_order_reviews_dataset.csv')
df_olist_order_items= pd.read_csv('/content/olist_order_items_dataset.csv')
df_olist_products= pd.read_csv('/content/olist_products_dataset.csv')
df_olist_geolocation= pd.read_csv('/content/olist_geolocation_dataset.csv')
df_product_category_name_translation= pd.read_csv('/content/product_category_name_translation.csv')
df_olist_orders = pd.read_csv('/content/olist_orders_dataset.csv')
df_olist_order_payments= pd.read_csv('/content/olist_order_payments_dataset.csv')

df_olist_customers.head()

from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

# export the dataframe as a table 'playstore' to the sqlite engine
df_olist_customers.to_sql("olist_customers", con =engine)
df_olist_sellers.to_sql("olist_sellers", con =engine)
df_olist_order_reviews.to_sql("olist_order_reviews", con =engine)
df_olist_order_items.to_sql("olist_order_items", con =engine)
df_olist_products.to_sql("olist_products_dataset", con =engine)
df_olist_geolocation.to_sql("olist_geolocation", con =engine)
df_product_category_name_translation.to_sql("product_category_name_translation", con =engine)
df_olist_orders.to_sql("olist_orders", con =engine)
df_olist_order_payments.to_sql("olist_order_payments", con =engine)
df_olist_order_payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [ ]:
sql='''

Select * from olist_customers
limit 5


''';


df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,index,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [ ]:
import pandas as pd
from sqlalchemy import create_engine, inspect

engine = create_engine("sqlite:////content/olist.db", echo=False)

files = {
    "olist_customers": "/content/olist_customers_dataset.csv",
    "olist_sellers": "/content/olist_sellers_dataset.csv",
    "olist_order_reviews": "/content/olist_order_reviews_dataset.csv",
    "olist_order_items": "/content/olist_order_items_dataset.csv",
    "olist_products": "/content/olist_products_dataset.csv",
    "olist_geolocation": "/content/olist_geolocation_dataset.csv",
    "product_category_name_translation": "/content/product_category_name_translation.csv",
    "olist_orders": "/content/olist_orders_dataset.csv",
    "olist_order_payments": "/content/olist_order_payments_dataset.csv",
}


for name, path in files.items():
    pd.read_csv(path, low_memory=False).to_sql(name, con=engine, if_exists="replace", index=False)



База данных создана → /content/olist.db

Таблицы в базе:
['olist_customers', 'olist_geolocation', 'olist_order_items', 'olist_order_payments', 'olist_order_reviews', 'olist_orders', 'olist_products', 'olist_sellers', 'product_category_name_translation']

Первые 5 строк из таблицы olist_orders:
                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
2    delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   
3    deliv

In [ ]:
query = """
SELECT
    COUNT(*) AS total_jan_orders,
    SUM(CASE WHEN r.review_score = 5 THEN 1 ELSE 0 END) AS jan_orders_score_5,
    ROUND(
        SUM(CASE WHEN r.review_score = 5 THEN 1 ELSE 0 END) * 100.0
        / COUNT(*),
        2
    ) AS percentage_score_5
FROM olist_orders o
JOIN olist_order_reviews r
    ON o.order_id = r.order_id
WHERE strftime('%Y-%m', o.order_purchase_timestamp) = '2018-01';
"""

pd.read_sql(query, con=engine)

,total_jan_orders,jan_orders_score_5,percentage_score_5
0,7245,4097,56.55


In [ ]:
# --- Query 2: Customer Purchase Trend Year-on-Year (YoY) ---

query = """
WITH yearly AS (
    SELECT
        -- Extract purchase year from timestamp
        CAST(strftime('%Y', o.order_purchase_timestamp) AS INTEGER) AS year,

        -- Total number of orders placed in this year
        COUNT(*) AS orders,

        -- Number of unique customers who made purchases in this year
        COUNT(DISTINCT o.customer_id) AS unique_customers
    FROM olist_orders o
    GROUP BY 1
)

SELECT
    year,
    orders,

    -- Orders from the previous year (LAG = previous row in window function)
    LAG(orders) OVER (ORDER BY year) AS orders_prev_year,

    -- YoY % change in the number of orders
    ROUND(
        (orders - LAG(orders) OVER (ORDER BY year)) * 100.0
        / NULLIF(LAG(orders) OVER (ORDER BY year), 0),
        2
    ) AS orders_yoy_pct,

    unique_customers,

    -- Unique customers from the previous year
    LAG(unique_customers) OVER (ORDER BY year) AS customers_prev_year,

    -- YoY % change in the number of customers
    ROUND(
        (unique_customers - LAG(unique_customers) OVER (ORDER BY year)) * 100.0
        / NULLIF(LAG(unique_customers) OVER (ORDER BY year), 0),
        2
    ) AS customers_yoy_pct,

    -- Average number of orders per customer for this year
    ROUND(1.0 * orders / unique_customers, 2) AS avg_orders_per_customer

FROM yearly
ORDER BY year;
"""

# Execute query and display the result
yoy = pd.read_sql(query, con=engine)
yoy

,year,orders,orders_prev_year,orders_yoy_pct,unique_customers,customers_prev_year,customers_yoy_pct,avg_orders_per_customer
0,2016,329,NaN,NaN,329,NaN,NaN,1.0
1,2017,45101,329.0,13608.51,45101,329.0,13608.51,1.0
2,2018,54011,45101.0,19.76,54011,45101.0,19.76,1.0


In [ ]:
query = """
SELECT
    o.customer_id,

    -- Total value of all items the customer has ever purchased
    SUM(oi.price + oi.freight_value) AS total_spent,

    -- Number of orders placed by this customer
    COUNT(DISTINCT o.order_id) AS total_orders,

    -- Average Order Value (AOV)
    ROUND(
        SUM(oi.price + oi.freight_value) * 1.0 /
        COUNT(DISTINCT o.order_id),
        2
    ) AS avg_order_value

FROM olist_orders o
JOIN olist_order_items oi
    ON o.order_id = oi.order_id

GROUP BY o.customer_id
ORDER BY avg_order_value DESC;
"""

aov = pd.read_sql(query, con=engine)
aov.head()

,customer_id,total_spent,total_orders,avg_order_value
0,1617b1357756262bfa56ab541c47bc16,13664.08,1,13664.08
1,ec5b2ba62e574342386871631fafd3fc,7274.88,1,7274.88
2,c6e2731c5b391845f6800c97401a43a9,6929.31,1,6929.31
3,f48d464a0baaea338cb25f816991ab1f,6922.21,1,6922.21
4,3fd6777bbce08a352fddd04e4a7cc8f6,6726.66,1,6726.66


In [ ]:
query = """
SELECT
    c.customer_city AS city,

    -- Total revenue generated by all customers in this city
    SUM(oi.price + oi.freight_value) AS total_revenue

FROM olist_orders o
JOIN olist_customers c
    ON o.customer_id = c.customer_id
JOIN olist_order_items oi
    ON o.order_id = oi.order_id

-- Only include orders placed between 2016 and 2018
WHERE CAST(strftime('%Y', o.order_purchase_timestamp) AS INTEGER) BETWEEN 2016 AND 2018

GROUP BY c.customer_city
ORDER BY total_revenue DESC
LIMIT 5;
"""

top_cities = pd.read_sql(query, con=engine)
top_cities

,city,total_revenue
0,sao paulo,2170227.12
1,rio de janeiro,1154234.02
2,belo horizonte,416733.39
3,brasilia,352305.14
4,curitiba,244739.87


In [ ]:
# State-wise revenue between 2016 and 2018 (inclusive).
# Revenue = SUM(price + freight_value).
# Output columns:
#   state, revenue_2016, revenue_2017, revenue_2018, total_revenue
# Sorted by total_revenue (desc).

query = """
SELECT
    c.customer_state AS state,

    -- Yearly revenue slices
    ROUND(SUM(CASE WHEN CAST(strftime('%Y', o.order_purchase_timestamp) AS INTEGER) = 2016
                   THEN (oi.price + oi.freight_value) ELSE 0 END), 2) AS revenue_2016,
    ROUND(SUM(CASE WHEN CAST(strftime('%Y', o.order_purchase_timestamp) AS INTEGER) = 2017
                   THEN (oi.price + oi.freight_value) ELSE 0 END), 2) AS revenue_2017,
    ROUND(SUM(CASE WHEN CAST(strftime('%Y', o.order_purchase_timestamp) AS INTEGER) = 2018
                   THEN (oi.price + oi.freight_value) ELSE 0 END), 2) AS revenue_2018,

    -- Total across 2016–2018
    ROUND(SUM(oi.price + oi.freight_value), 2) AS total_revenue

FROM olist_orders o
JOIN olist_customers c
    ON o.customer_id = c.customer_id
JOIN olist_order_items oi
    ON o.order_id = oi.order_id

WHERE CAST(strftime('%Y', o.order_purchase_timestamp) AS INTEGER) BETWEEN 2016 AND 2018

GROUP BY c.customer_state
ORDER BY total_revenue DESC;
"""

state_revenue = pd.read_sql(query, con=engine)
state_revenue

,state,revenue_2016,revenue_2017,revenue_2018,total_revenue
0,SP,16629.80,2513485.93,3391562.39,5921678.12
1,RJ,12347.60,1047113.24,1070221.14,2129681.98
2,MG,5620.55,842659.58,1007881.36,1856161.49
3,RS,4712.97,420892.05,460221.74,885826.76
4,PR,2494.72,345195.33,453245.39,800935.44
5,BA,995.34,281062.53,329448.80,611506.67
6,SC,2654.23,271785.25,335774.12,610213.60
7,DF,1200.11,155786.30,196243.03,353229.44
8,GO,1154.67,162778.57,183773.69,347706.93
9,ES,1067.14,142252.67,181482.10,324801.91


In [ ]:
query = """
WITH base AS (
    SELECT
        oi.seller_id,
        o.order_id,
        o.customer_id,
        oi.price,
        oi.freight_value,
        r.review_score,
        -- Extract year for optional filtering
        CAST(strftime('%Y', o.order_purchase_timestamp) AS INTEGER) AS order_year
    FROM olist_order_items AS oi
    JOIN olist_orders AS o
      ON oi.order_id = o.order_id
    LEFT JOIN olist_order_reviews AS r
      ON o.order_id = r.order_id
    -- WHERE order_year BETWEEN 2016 AND 2018
),
seller_kpi AS (
    SELECT
        seller_id,
        COUNT(*) AS goods_sold,                                        -- items sold
        SUM(price + freight_value) AS total_revenue,                   -- revenue
        COUNT(DISTINCT customer_id) AS unique_customers,               -- buyer count
        SUM(CASE WHEN review_score = 5 THEN 1 ELSE 0 END) AS five_star_reviews,  -- 5★ count
        ROUND(
            100.0 * SUM(CASE WHEN review_score = 5 THEN 1 ELSE 0 END)
            / NULLIF(COUNT(DISTINCT order_id), 0), 2
        ) AS five_star_share_pct                                       -- optional: 5★ share of reviewed orders
    FROM base
    GROUP BY seller_id
),
ranked AS (
    SELECT
        seller_id,
        goods_sold,
        total_revenue,
        unique_customers,
        five_star_reviews,
        five_star_share_pct,
        -- Rank each KPI (higher is better)
        DENSE_RANK() OVER (ORDER BY goods_sold DESC)        AS r_goods,
        DENSE_RANK() OVER (ORDER BY total_revenue DESC)     AS r_revenue,
        DENSE_RANK() OVER (ORDER BY unique_customers DESC)  AS r_customers,
        DENSE_RANK() OVER (ORDER BY five_star_reviews DESC) AS r_5star
    FROM seller_kpi
),
overall AS (
    SELECT
        seller_id,
        goods_sold,
        ROUND(total_revenue, 2) AS total_revenue,
        unique_customers,
        five_star_reviews,
        five_star_share_pct,
        r_goods, r_revenue, r_customers, r_5star,
        -- Simple composite score: lower = better (sum of ranks)
        (r_goods + r_revenue + r_customers + r_5star) AS composite_rank
    FROM ranked
)
SELECT
    seller_id,
    goods_sold,
    total_revenue,
    unique_customers,
    five_star_reviews,
    five_star_share_pct,
    r_goods, r_revenue, r_customers, r_5star,
    composite_rank
FROM overall
ORDER BY composite_rank ASC, total_revenue DESC
LIMIT 20;   -- top overall sellers by combined success
"""

top_sellers = pd.read_sql(query, con=engine)
top_sellers

,seller_id,goods_sold,total_revenue,unique_customers,five_star_reviews,five_star_share_pct,r_goods,r_revenue,r_customers,r_5star,composite_rank
0,4a3ca9315b744ce9f8e9374361493884,2009,238440.31,1806,947,52.44,2,3,2,4,11
1,6560211a19b47992c3666cc44a7e94c0,2039,151636.13,1854,1024,55.23,1,11,1,3,16
2,1f50f920176fa81dab994f9023523100,1940,142757.03,1404,1096,78.06,3,12,4,1,20
3,da8622b14eb17ae2831f4ac5b9dab84a,1574,188062.51,1314,893,67.96,5,6,5,5,21
4,cc419e0650a3c5ba77189a1882b7556a,1819,132877.06,1706,1053,61.72,4,15,3,2,24
5,955fee9216a65b617aa5c0531780ce60,1501,160694.86,1287,804,62.47,6,10,6,6,28
6,4869f7a5dfa277a7dca6462dcf3b52b2,1156,249640.70,1132,683,60.34,11,1,9,9,30
7,7a67c85e85bb2ce8582c35f2203ad736,1175,163278.78,1160,717,61.81,10,9,7,8,34
8,1025f0e2d44d7041d6cf58b6550e0bfa,1443,174710.82,915,729,79.67,7,8,13,7,35
9,7c67e1448b00f6e969d365cea6b010ab,1375,241374.82,982,437,44.50,8,2,11,16,37


In [ ]:
query = """
SELECT
    c.customer_state AS state,

    COUNT(DISTINCT o.order_id) AS total_orders,                                   -- all orders placed by customers in this state
    SUM(CASE WHEN o.order_status = 'delivered' THEN 1 ELSE 0 END) AS delivered_orders, -- orders marked as delivered

    ROUND(
        100.0 * SUM(CASE WHEN o.order_status = 'delivered' THEN 1 ELSE 0 END)
        / NULLIF(COUNT(DISTINCT o.order_id), 0),
        2
    ) AS delivery_success_rate_pct                                                -- delivered / total * 100

FROM olist_orders o
JOIN olist_customers c
  ON o.customer_id = c.customer_id

-- Uncomment to focus on 2016–2018 only:
-- WHERE CAST(strftime('%Y', o.order_purchase_timestamp) AS INTEGER) BETWEEN 2016 AND 2018

GROUP BY c.customer_state

-- Optional: filter out states with very few orders (to avoid noisy percentages)
-- HAVING COUNT(DISTINCT o.order_id) >= 50

ORDER BY delivery_success_rate_pct DESC, total_orders DESC;
"""

delivery_success = pd.read_sql(query, con=engine)
delivery_success

,state,total_orders,delivered_orders,delivery_success_rate_pct
0,AC,81,80,98.77
1,AP,68,67,98.53
2,ES,2033,1995,98.13
3,MS,715,701,98.04
4,AM,148,145,97.97
5,TO,280,274,97.86
6,RS,5466,5345,97.79
7,RN,485,474,97.73
8,MT,907,886,97.68
9,MG,11635,11354,97.58


In [ ]:
query = """
WITH items_products AS (
    SELECT
        oi.order_id,
        -- Normalize category name; some rows may have NULL
        COALESCE(p.product_category_name, 'unknown') AS product_category_name
    FROM olist_order_items AS oi
    JOIN olist_products AS p
      ON oi.product_id = p.product_id
    -- Optional: restrict to certain years via orders table
    -- JOIN olist_orders AS o
    --   ON oi.order_id = o.order_id
    -- WHERE CAST(strftime('%Y', o.order_purchase_timestamp) AS INTEGER) BETWEEN 2016 AND 2018
),
category_payment AS (
    SELECT
        ip.product_category_name,
        pay.payment_type,
        -- Count distinct orders in which this payment type appeared for the category
        COUNT(DISTINCT ip.order_id) AS orders_with_type,
        -- Sum of payment values for those orders (handles split payments)
        SUM(pay.payment_value) AS total_paid_value
    FROM items_products AS ip
    JOIN olist_order_payments AS pay
      ON ip.order_id = pay.order_id
    GROUP BY ip.product_category_name, pay.payment_type
),
ranked AS (
    SELECT
        product_category_name,
        payment_type,
        orders_with_type,
        ROUND(total_paid_value, 2) AS total_paid_value,
        ROW_NUMBER() OVER (
            PARTITION BY product_category_name
            ORDER BY orders_with_type DESC, total_paid_value DESC
        ) AS rn
    FROM category_payment
)
SELECT
    product_category_name,
    payment_type AS preferred_payment_type,
    orders_with_type,
    total_paid_value
FROM ranked
WHERE rn = 1
ORDER BY product_category_name;
"""

preferred_payment_by_category = pd.read_sql(query, con=engine)
preferred_payment_by_category

,product_category_name,preferred_payment_type,orders_with_type,total_paid_value
0,agro_industria_e_comercio,credit_card,125,61173.89
1,alimentos,credit_card,328,36370.75
2,alimentos_bebidas,credit_card,164,20183.72
3,artes,credit_card,147,17460.66
4,artes_e_artesanato,credit_card,13,1365.82
...,...,...,...,...
69,tablets_impressao_imagem,credit_card,62,7862.80
70,telefonia,credit_card,3108,371512.51
71,telefonia_fixa,credit_card,162,147824.98
72,unknown,credit_card,1122,184300.89


In [ ]:
# --- Query 9: Distance Between Cities (Customer City ↔ Seller City) ---

import math
import pandas as pd
from sqlalchemy import text

# 1) Build / refresh city centroids (lat/lon) from geolocation table
#    We use geolocation_city/state to avoid relying on zip_code_prefix.
with engine.begin() as conn:
    conn.exec_driver_sql("DROP TABLE IF EXISTS city_centroids;")
    conn.exec_driver_sql("""
        CREATE TABLE city_centroids AS
        SELECT
            geolocation_city  AS city,
            geolocation_state AS state,
            AVG(geolocation_lat) AS lat,
            AVG(geolocation_lng) AS lon
        FROM olist_geolocation
        WHERE geolocation_city IS NOT NULL
          AND geolocation_state IS NOT NULL
        GROUP BY geolocation_city, geolocation_state;
    """)

# 2) Register a Python -> SQLite UDF so we can call HAVERSINE inside SQL
#    (SQLite doesn't have trig/geo distance built-in by default)
def haversine_km(lat1, lon1, lat2, lon2):
    """Great-circle distance between two WGS84 points in kilometers."""
    if None in (lat1, lon1, lat2, lon2):
        return None
    # convert degrees to radians
    φ1 = math.radians(lat1); λ1 = math.radians(lon1)
    φ2 = math.radians(lat2); λ2 = math.radians(lon2)
    dφ = φ2 - φ1
    dλ = λ2 - λ1
    a = math.sin(dφ/2)**2 + math.cos(φ1) * math.cos(φ2) * math.sin(dλ/2)**2
    c = 2 * math.asin(math.sqrt(a))
    R = 6371.0088  # mean Earth radius (km)
    return R * c

# Attach the UDF to the underlying sqlite3 connection
raw = engine.raw_connection()
raw.create_function("haversine_km", 4, haversine_km)
raw.commit()
raw.close()

# 3) Compute city-to-city distance for every order-item, then aggregate by city pair
#    (You can uncomment the WHERE year filter to focus on 2016–2018)
query = """
WITH order_parties AS (
    SELECT
        o.order_id,
        -- customer city/state
        c.customer_city  AS cust_city,
        c.customer_state AS cust_state,
        -- seller city/state (via order_items)
        s.seller_city    AS sell_city,
        s.seller_state   AS sell_state,
        CAST(strftime('%Y', o.order_purchase_timestamp) AS INTEGER) AS order_year
    FROM olist_order_items oi
    JOIN olist_orders      o ON oi.order_id = o.order_id
    JOIN olist_customers   c ON o.customer_id = c.customer_id
    JOIN olist_sellers     s ON oi.seller_id = s.seller_id
),
coords AS (
    SELECT
        op.order_id,
        op.cust_city, op.cust_state,
        op.sell_city, op.sell_state,
        cc.lat  AS cust_lat,  cc.lon  AS cust_lon,
        sc.lat  AS sell_lat,  sc.lon  AS sell_lon,
        op.order_year
    FROM order_parties op
    LEFT JOIN city_centroids cc
           ON cc.city = op.cust_city AND cc.state = op.cust_state
    LEFT JOIN city_centroids sc
           ON sc.city = op.sell_city AND sc.state = op.sell_state
    -- Uncomment to restrict to 2016–2018:
    -- WHERE op.order_year BETWEEN 2016 AND 2018
)
SELECT
    cust_city || ', ' || cust_state AS customer_city,
    sell_city || ', ' || sell_state AS seller_city,
    COUNT(*)                                   AS order_items_count,
    ROUND(AVG(haversine_km(cust_lat, cust_lon, sell_lat, sell_lon)), 2) AS avg_distance_km,
    ROUND(MIN(haversine_km(cust_lat, cust_lon, sell_lat, sell_lon)), 2) AS min_distance_km,
    ROUND(MAX(haversine_km(cust_lat, cust_lon, sell_lat, sell_lon)), 2) AS max_distance_km
FROM coords
WHERE cust_lat IS NOT NULL AND cust_lon IS NOT NULL
  AND sell_lat IS NOT NULL AND sell_lon IS NOT NULL
GROUP BY customer_city, seller_city
HAVING order_items_count >= 10         -- optional: keep frequent pairs only (reduce noise)
ORDER BY avg_distance_km DESC
LIMIT 20;                              -- show the 20 longest average city-to-city pairs
"""

city_distances = pd.read_sql(text(query), con=engine)
city_distances

,customer_city,seller_city,order_items_count,avg_distance_km,min_distance_km,max_distance_km
0,"boa vista, RR","sao paulo, SP",17,3306.78,3306.78,3306.78
1,"florianopolis, SC","sao luis, MA",13,2821.68,2821.68,2821.68
2,"rio branco, AC","sao paulo, SP",20,2710.03,2710.03,2710.03
3,"manaus, AM","sao paulo, SP",59,2694.26,2694.26,2694.26
4,"belem, PA","curitiba, PR",10,2673.95,2673.95,2673.95
5,"fortaleza, CE","curitiba, PR",18,2670.85,2670.85,2670.85
6,"macapa, AP","sao paulo, SP",21,2668.21,2668.21,2668.21
7,"sao luis, MA","curitiba, PR",12,2604.90,2604.90,2604.90
8,"joao pessoa, PB","curitiba, PR",15,2548.25,2548.25,2548.25
9,"santarem, PA","sao paulo, SP",14,2506.00,2506.00,2506.00
